In [1]:
import pandas as pd
train_text = pd.read_json("https://bit.ly/nlp-tweet-train",lines=True)
test_text =pd.read_json("https://bit.ly/nlp-tweet-test",lines=True)

In [2]:
#text['text']

In [3]:
from collections import Counter
from collections import defaultdict
import nltk
import re
import numpy as np
import math

### Part1

In [4]:
from nltk.tokenize import TweetTokenizer
import re
tokenizer = TweetTokenizer()
def get_corpus(text):
    cp =[]
    for sentence in text['text']:
        corpus =[token for token in re.split('\W+',sentence.lower()) if token !='']
        cp.append(corpus)
    return cp

#train_tokens = [tokenizer.tokenize(re.split('\W+',sentence.lower()))for sentence in train_text['text'] ]
#test_tokens = [tokenizer.tokenize(re.split('\W+',sentence.lower()))for sentence in test_text['text'] ]
train_tokens = get_corpus(train_text)
test_tokens = get_corpus(test_text)

In [5]:
def get_vocab(tokens):
    vocab = Counter()
    for k in tokens:
        vocab.update(k)
    vocab = [token if freq>2 else "<UNK>" for token,freq in vocab.most_common()]
    return vocab
vocab_train = get_vocab(train_tokens)
vocab_test = get_vocab(test_tokens)

In [6]:
def get_bigram(corpus_token,vocab):
    corpus_bigrams=[]
    tmp = corpus_token
    for i in range(len(corpus_token)):
        for j in range(len(corpus_token[i])):
            if corpus_token[i][j] not in vocab:
                #print(corpus_token[i][j])
                tmp[i][j] = "<UNK>"
        corpus_bigrams.append(list(nltk.bigrams(['<s>'] + tmp[i] +['</s>'])))
    return corpus_bigrams
train_bigram = get_bigram(train_tokens,vocab_train)
test_bigram = get_bigram(test_tokens,vocab_test)

In [7]:
def get_dictionary(train_bigram):
    counts = defaultdict(lambda: defaultdict(lambda:0))
    for bi_tok in train_bigram:
        for w1,w2 in bi_tok:
        #print(w1,w2)
            counts[w1][w2]+=1
    return counts
train_counts = get_dictionary(train_bigram)
test_counts = get_dictionary(test_bigram)
#[counts[w1][w2] for w1,w2 in sentence_bigrams]

In [8]:
train_count =[[1+train_counts[w1][w2] for w1,w2 in bi_gram ]for bi_gram in train_bigram]
train_counts_sum=[[len(vocab_train)+sum(train_counts[w1].values()) for w1,w2 in bi_gram ]for bi_gram in train_bigram]

In [9]:
#Used test_bigram
test_count =[[1+test_counts[w1][w2] for w1,w2 in bi_gram ]for bi_gram in test_bigram]
test_counts_sum=[[len(vocab_test)+sum(test_counts[w1].values()) for w1,w2 in bi_gram ]for bi_gram in test_bigram]

In [ ]:
#Used train_bigram
test_count =[[1+train_counts[w1][w2] for w1,w2 in bi_gram ]for bi_gram in test_bigram]
test_counts_sum=[[len(vocab_train)+sum(train_counts[w1].values()) for w1,w2 in bi_gram ]for bi_gram in test_bigram]

In [ ]:
def PPL(count,count_sum,bigram):
    prob = [np.divide(x,y) for x,y in zip(count,count_sum)]
    PP_list =[]
    for i in range(len(prob)):
        N = len(bigram[i])
        tmp = np.prod(prob[i])
        if tmp ==0:
        #print('here')
            tmp = 1e-200
        a = math.pow(tmp,-1/N)
        PP_list.append(a)
    return np.mean(PP_list)
print("train_AVG_PPL:{}".format(PPL(train_count,train_counts_sum,train_bigram)))
print("test_AVG_PPL:{}".format(PPL(test_count,test_counts_sum,test_bigram)))
#[sum(counts[w1].values()) for w1,w2 in sentence_bigrams]

### Part2

In [11]:
def get_bi_dict(train_bigram):
    counts_fw = defaultdict(lambda: defaultdict(lambda:0))
    counts_bw =  defaultdict(lambda: defaultdict(lambda:0))
    for bi_tok in train_bigram:
        for w1,w2 in bi_tok:
        #print(w1,w2)
            counts_fw[w1][w2]+=1
        for w1,w2 in bi_tok:
            counts_bw[w2][w1]+=1
    return counts_fw,counts_bw
ct_fw,ct_bw = get_bi_dict(train_bigram)
ctt_fw,ctt_bw = get_bi_dict(test_bigram)

In [12]:
test_counts_fw =[[1+ct_fw[w1][w2] for w1,w2 in bi_gram ]for bi_gram in test_bigram]

In [13]:
test_counts_bw =[[1+ct_bw[w2][w1] for w1,w2 in bi_gram ]for bi_gram in test_bigram]

In [14]:
print(test_counts_bw[0],test_counts_fw[0])

[107, 52, 1, 3, 109, 312, 1, 1, 1, 9705, 32, 4, 115, 2, 1, 171, 132, 109, 9, 4, 165] [107, 52, 1, 3, 109, 312, 1, 1, 1, 9705, 32, 4, 115, 2, 1, 171, 132, 109, 9, 4, 165]


In [15]:
train_fw_sum = [[(1 + ct_fw[bi_gram[i][0]][bi_gram[i][1]])/(len(vocab_train) + sum(ct_fw[bi_gram[i][0]].values())) for i in range(len(bi_gram)-1) ]for bi_gram in train_bigram]
train_bw_sum = [[(1 + ct_bw[bi_gram[i][1]][bi_gram[i][0]])/(len(vocab_train) + sum(ct_bw[bi_gram[i][1]].values())) for i in range(1,len(bi_gram)) ]for bi_gram in train_bigram]

In [16]:
# test used test_bigram
#test_fw_sum=[[len(vocab)+sum(ct_fw[w1].values()) for w1,w2 in bi_gram ]for bi_gram in test_bigram]
test_fw_sum = [[(1 + ctt_fw[bi_gram[i][0]][bi_gram[i][1]])/(len(vocab_test) + sum(ctt_fw[bi_gram[i][0]].values())) for i in range(len(bi_gram)-1) ]for bi_gram in test_bigram]
test_bw_sum = [[(1 + ctt_bw[bi_gram[i][1]][bi_gram[i][0]])/(len(vocab_test) + sum(ctt_bw[bi_gram[i][1]].values())) for i in range(1,len(bi_gram)) ]for bi_gram in test_bigram]

In [25]:
# used train_bigram
test_fw_sum = [[(1 + ct_fw[bi_gram[i][0]][bi_gram[i][1]])/(len(vocab_train) + sum(ct_fw[bi_gram[i][0]].values())) for i in range(len(bi_gram)-1) ]for bi_gram in test_bigram]
test_bw_sum = [[(1 + ct_bw[bi_gram[i][1]][bi_gram[i][0]])/(len(vocab_train) + sum(ct_bw[bi_gram[i][1]].values())) for i in range(1,len(bi_gram)) ]for bi_gram in test_bigram]

In [26]:
def get_ppl(fw,bw,gamma,bigram):
    prob =[]
    PP_list=[]
    #gamma =np.arange(0,1,0.05)
    for i in range(len(bigram)):
        N = len(bigram[i])
        #print(fw[i],gamma,len(bw[i]),len(test_sum[i]))
        tmp = np.prod(list(map(lambda x,y:x*gamma+y*(1-gamma),fw[i],bw[i])))
        #tmp = np.prod(np.divide(list(map(lambda x,y: x * gamma+y*(1-gamma), fw[i],bw[i])),test_sum[i]))
        #print(tmp)
        if tmp ==0:
        #print('here')
            tmp = 1e-200
        a = math.pow(tmp,-1/N)
        PP_list.append(a)
        #prob.append(np.divide(fw[i]*gamma+bw[i]*(1-gamma),test_sum[i]))
    return np.mean(PP_list)

In [27]:
def get_gamma(fw,bw,bigram):
    gammas = np.arange(0,1.05,0.05)
    tmp = math.pow(10,15)
    tmp_g = 0
    for gamma in gammas:
        if tmp>get_ppl(fw,bw,gamma,bigram):
            tmp = get_ppl(fw,bw,gamma,bigram)
            tmp_g = gamma
        #tmp_small = get_ppl

        print(get_ppl(fw,bw,gamma,bigram))
        print(int(gamma*100)/100)
        #return tmp,tmp_g
    return tmp,tmp_g
best_train,best_train_gamma = get_gamma(train_fw_sum,train_bw_sum,train_bigram)
best_test,best_test_gamma = get_gamma(test_fw_sum,test_bw_sum,test_bigram)

2525.6197933675326
0.0
2030.9389852457593
0.05
1867.918942934626
0.1
1768.6487733650297
0.15
1700.1278816735107
0.2
1650.273794963705
0.25
1613.2807572861993
0.3
1585.9551232277859
0.35
1566.414492964196
0.4
1553.5317129130385
0.45
1546.6727647164112
0.5
1545.5732151249833
0.55
1550.2947989234303
0.6
1561.244771563732
0.65
1579.2680511245112
0.7
1605.8586674635587
0.75
1643.6196019546883
0.8
1697.3507963698917
0.85
1777.1421783228243
0.9
1910.9146876538598
0.95
2348.3541019291747
1.0
2817.2247230517305
0.0
2155.8666815049687
0.05
1957.5534521783004
0.1
1839.4153762791261
0.15
1759.0246437777466
0.2
1701.1826552228345
0.25
1658.6894324031596
0.3
1627.6269933122865
0.35
1605.7070603255365
0.4
1591.5719644765154
0.45
1584.467922766488
0.5
1584.0925472450078
0.55
1590.5427217395531
0.6
1604.3413538577631
0.65
1626.5565609974797
0.7
1659.0741998895235
0.75
1705.1930821929004
0.8
1771.0448218091683
0.85
1869.678334428886
0.9
2037.9290736209516
0.95
2642.629004041137
1.0


In [28]:
print("In train_data set best_of bidirection_ppl:{},and it's gamma:{}".format(best_train,best_train_gamma))
print("In test_data  set best_of bidirection_ppl:{},and it's gamma:{}".format(best_test,best_test_gamma))

In train_data set best_of bidirection_ppl:1545.5732151249833,and it's gamma:0.55
In test_data  set best_of bidirection_ppl:1584.0925472450078,and it's gamma:0.55


In [20]:
np.arange(0,1.05,0.05)

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

In [21]:
[1]+[2]

[1, 2]

In [22]:
print(len(test_counts_bw[0]),len(test_counts_fw[0]))

21 21


In [23]:
print(test_counts_bw[0])
print(test_bigram[0][-8])
print(ct_bw['</s>']['.'])

[107, 52, 1, 3, 109, 312, 1, 1, 1, 9705, 32, 4, 115, 2, 1, 171, 132, 109, 9, 4, 165]
('your', 'spending')
0


In [24]:
rs=[]
for bi_gram in test_bigram[:2]:
    ct=[]
    print(bi_gram)
    for i in range(len(bi_gram)):
        print(bi_gram[i][1],bi_gram[i][0])
        print(ct_bw[bi_gram[i][1]][bi_gram[i][0]])
        print("____________")
        print(ct_fw[bi_gram[i][0]][bi_gram[i][1]])
        ct.append(1+ct_bw[bi_gram[i][1]][bi_gram[i][0]])
    print(ct)
    rs.append(ct)
print(rs[0])
#test used really not used 無用

[('<s>', 'more'), ('more', 'money'), ('money', 'wasted'), ('wasted', 'on'), ('on', 'an'), ('an', '<UNK>'), ('<UNK>', 'antiquated'), ('antiquated', 'airport'), ('airport', 'it'), ('it', 's'), ('s', 'easy'), ('easy', 'when'), ('when', 'your'), ('your', 'spending'), ('spending', 'other'), ('other', 'people'), ('people', 's'), ('s', 'hard'), ('hard', 'earned'), ('earned', 'money'), ('money', '</s>')]
more <s>
106
____________
106
money more
51
____________
51
wasted money
0
____________
0
on wasted
2
____________
2
an on
108
____________
108
<UNK> an
311
____________
311
antiquated <UNK>
0
____________
0
airport antiquated
0
____________
0
it airport
0
____________
0
s it
9704
____________
9704
easy s
31
____________
31
when easy
3
____________
3
your when
114
____________
114
spending your
1
____________
1
other spending
0
____________
0
people other
170
____________
170
s people
131
____________
131
hard s
108
____________
108
earned hard
8
____________
8
money earned
3
____________
3
</